In [1]:
import pandas as pd
#from keras.preprocessing import image
from matplotlib import pyplot
from matplotlib.pyplot import imshow
import split_data as sd
import basic_model as n
import numpy as np
from sklearn.metrics import confusion_matrix
#import Network as net
import vectorised as v
from random import randint
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras.backend.tensorflow_backend import set_session
set_session(tf.Session(config=config))
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
config=tf.ConfigProto()
config.gpu_options.allow_growth=True

Using TensorFlow backend.


In [2]:
data=pd.read_csv('/home/snehao/dog_breed/Image_Classification/labels.csv')
print(data.head())
targets_series = pd.Series(data['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels = np.asarray(one_hot)

                                 id             breed
0  000bec180eb18c7604dcecc8fe0dba07       boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97             dingo
2  001cdf01b096e06d78e9e5112d419397          pekinese
3  00214f311d5d2247d5dfe4fe24b2303d          bluetick
4  0021f9ceb3235effd7fcde7f7538ed62  golden_retriever


In [3]:
x_train,x_test,y_train,y_test=sd.split_data(data,one_hot_labels,.10)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(9199, 2)
(9199, 120)
(1023, 2)
(1023, 120)


In [4]:
x=[]
y=[]
#for i in range(100):
for i in range(x_train.shape[0]):
    img = image.load_img('/home/snehao/dog_breed/train/'+x_train.iloc[i][0]+'.jpg',target_size=(300,300,3),interpolation='bicubic')
    arr=image.img_to_array(img)
    x.append(arr/255)
    y.append(y_train[i])
x=np.array(x)
y=np.array(y)
print('x_shape',x.shape)
print('y_shape',y.shape)

x_shape (9199, 300, 300, 3)
y_shape (9199, 120)


In [5]:
import cv2
x_val_images=[]
y_val1=[]
#for i in range(100):
for i in range(x_test.shape[0]): 
    
    img = image.load_img('/home/snehao/dog_breed/train/'+x_test.iloc[i][0]+'.jpg',target_size=(300,300,3),interpolation='bicubic')
    arr=image.img_to_array(img)
    x_val_images.append(arr/255)
    y_val1.append(y_test[i])
x_val_images=np.array(x_val_images)
y_val1=np.array(y_val1)
print('y_shape',y_val1.shape)
print('x_shape',x_val_images.shape)


y_shape (1023, 120)
x_shape (1023, 300, 300, 3)


In [18]:
from keras.layers import Concatenate,Add,BatchNormalization,Input,Dropout,Dense,Activation,Flatten,MaxPooling2D,Conv2D,ZeroPadding2D
from keras.models import Model,Sequential
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
#base_model = Xception(input_shape=(300,300,3),weights='imagenet', include_top=False,pooling='avg')
base_model = InceptionV3(input_shape=(300,300,3),weights='imagenet', include_top=False,pooling='avg')

In [ ]:
'''import numpy as np
xtrain=np.load('xceptiontrainx.npy')
ytrain=np.load('xceptiontrainy.npy')'''

In [ ]:
'''xval=np.load('xceptionvalx.npy')
yval=np.load('xceptionvaly.npy')'''

In [19]:
callback=keras.callbacks.TensorBoard(log_dir='inception_graph', histogram_freq=0,  
          write_graph=True, write_images=True)

In [22]:
callback1=keras.callbacks.ModelCheckpoint('inceptionpoint31.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [7]:
featuresx=base_model.predict(x)

In [8]:
features_val=base_model.predict(x_val_images)


In [9]:
featuresx.shape

(9199, 2048)

In [23]:
from keras import optimizers
input_shape=Input((2048,)) 
x1=Dense(1024, activation='relu')(input_shape) 
#x1=Dropout(0.4)(x1) 
#x1=Dense(512, activation='relu')(x1) 
#x1=Dropout(0.2)(x1) 
x1=Dense(120, activation='softmax')(x1) 
model=Model(inputs=input_shape,outputs=x1) 
print(model.summary()) 




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_6 (Dense)              (None, 120)               123000    
Total params: 2,221,176
Trainable params: 2,221,176
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
sgd=optimizers.SGD( momentum=0.0, decay=0.0, nesterov=False) 
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [25]:
model.fit(featuresx,y,epochs=250,validation_data=(features_val,y_val1),batch_size=150,callbacks=[callback,callback1])

Train on 9199 samples, validate on 1023 samples
Epoch 1/250
9199/9199 [==============================] - 1s 115us/step - loss: 4.6564 - acc: 0.0535 - val_loss: 4.4128 - val_acc: 0.1574
Epoch 2/250
9199/9199 [==============================] - 1s 70us/step - loss: 4.1979 - acc: 0.3084 - val_loss: 3.9715 - val_acc: 0.4692
Epoch 3/250
9199/9199 [==============================] - 1s 72us/step - loss: 3.7276 - acc: 0.5572 - val_loss: 3.4921 - val_acc: 0.6403
Epoch 4/250
9199/9199 [==============================] - 1s 90us/step - loss: 3.2263 - acc: 0.6919 - val_loss: 2.9823 - val_acc: 0.7195
Epoch 5/250
9199/9199 [==============================] - 1s 89us/step - loss: 2.7165 - acc: 0.7650 - val_loss: 2.4801 - val_acc: 0.7693
Epoch 6/250
9199/9199 [==============================] - 1s 126us/step - loss: 2.2368 - acc: 0.8065 - val_loss: 2.0271 - val_acc: 0.8152
Epoch 7/250
9199/9199 [==============================] - 1s 130us/step - loss: 1.8199 - acc: 0.8439 - val_loss: 1.6525 - val_acc: 0.84

9199/9199 [==============================] - 1s 128us/step - loss: 0.2340 - acc: 0.9387 - val_loss: 0.3260 - val_acc: 0.8983
Epoch 61/250
9199/9199 [==============================] - 1s 122us/step - loss: 0.2319 - acc: 0.9387 - val_loss: 0.3255 - val_acc: 0.8974
Epoch 62/250
9199/9199 [==============================] - 1s 92us/step - loss: 0.2299 - acc: 0.9392 - val_loss: 0.3233 - val_acc: 0.9003
Epoch 63/250
9199/9199 [==============================] - 0s 50us/step - loss: 0.2280 - acc: 0.9392 - val_loss: 0.3243 - val_acc: 0.8964
Epoch 64/250
9199/9199 [==============================] - 1s 71us/step - loss: 0.2260 - acc: 0.9408 - val_loss: 0.3235 - val_acc: 0.8964
Epoch 65/250
9199/9199 [==============================] - 0s 47us/step - loss: 0.2242 - acc: 0.9401 - val_loss: 0.3241 - val_acc: 0.8964
Epoch 66/250
9199/9199 [==============================] - 1s 76us/step - loss: 0.2224 - acc: 0.9413 - val_loss: 0.3215 - val_acc: 0.8964
Epoch 67/250
9199/9199 [============================

Epoch 120/250
9199/9199 [==============================] - 1s 77us/step - loss: 0.1561 - acc: 0.9633 - val_loss: 0.3046 - val_acc: 0.8974
Epoch 121/250
9199/9199 [==============================] - 0s 31us/step - loss: 0.1555 - acc: 0.9620 - val_loss: 0.3073 - val_acc: 0.8944
Epoch 122/250
9199/9199 [==============================] - 1s 65us/step - loss: 0.1546 - acc: 0.9624 - val_loss: 0.3084 - val_acc: 0.8964
Epoch 123/250
9199/9199 [==============================] - 1s 92us/step - loss: 0.1538 - acc: 0.9639 - val_loss: 0.3071 - val_acc: 0.8915
Epoch 124/250
9199/9199 [==============================] - ETA: 0s - loss: 0.1543 - acc: 0.963 - 1s 56us/step - loss: 0.1529 - acc: 0.9637 - val_loss: 0.3066 - val_acc: 0.8944
Epoch 125/250
9199/9199 [==============================] - 1s 75us/step - loss: 0.1521 - acc: 0.9634 - val_loss: 0.3056 - val_acc: 0.8974
Epoch 126/250
9199/9199 [==============================] - 0s 46us/step - loss: 0.1513 - acc: 0.9640 - val_loss: 0.3048 - val_acc: 0.8

9199/9199 [==============================] - 1s 77us/step - loss: 0.1171 - acc: 0.9749 - val_loss: 0.3057 - val_acc: 0.8944
Epoch 180/250
9199/9199 [==============================] - 0s 47us/step - loss: 0.1164 - acc: 0.9760 - val_loss: 0.3058 - val_acc: 0.8944
Epoch 181/250
9199/9199 [==============================] - 0s 34us/step - loss: 0.1162 - acc: 0.9759 - val_loss: 0.3049 - val_acc: 0.8954
Epoch 182/250
9199/9199 [==============================] - 1s 88us/step - loss: 0.1155 - acc: 0.9760 - val_loss: 0.3052 - val_acc: 0.8935
Epoch 183/250
9199/9199 [==============================] - 1s 87us/step - loss: 0.1151 - acc: 0.9754 - val_loss: 0.3060 - val_acc: 0.8905
Epoch 184/250
9199/9199 [==============================] - 1s 55us/step - loss: 0.1146 - acc: 0.9764 - val_loss: 0.3065 - val_acc: 0.8915
Epoch 185/250
9199/9199 [==============================] - 0s 53us/step - loss: 0.1139 - acc: 0.9762 - val_loss: 0.3056 - val_acc: 0.8935
Epoch 186/250
9199/9199 [=======================

9199/9199 [==============================] - 1s 66us/step - loss: 0.0920 - acc: 0.9836 - val_loss: 0.3077 - val_acc: 0.8974
Epoch 239/250
9199/9199 [==============================] - 1s 56us/step - loss: 0.0917 - acc: 0.9838 - val_loss: 0.3084 - val_acc: 0.8944
Epoch 240/250
9199/9199 [==============================] - 1s 85us/step - loss: 0.0913 - acc: 0.9830 - val_loss: 0.3100 - val_acc: 0.8944
Epoch 241/250
9199/9199 [==============================] - 0s 42us/step - loss: 0.0909 - acc: 0.9837 - val_loss: 0.3086 - val_acc: 0.8925
Epoch 242/250
9199/9199 [==============================] - 1s 87us/step - loss: 0.0906 - acc: 0.9840 - val_loss: 0.3099 - val_acc: 0.8925
Epoch 243/250
9199/9199 [==============================] - 0s 49us/step - loss: 0.0904 - acc: 0.9841 - val_loss: 0.3087 - val_acc: 0.8944
Epoch 244/250
9199/9199 [==============================] - 1s 73us/step - loss: 0.0900 - acc: 0.9845 - val_loss: 0.3090 - val_acc: 0.8925
Epoch 245/250
9199/9199 [=======================

In [ ]:
model.save_weights('inception1.h5')


In [ ]:
predics=model.predict(x_test_images)


